# Refactor the Taxi Fare Prediction Problem with a Pipeline

We will refactor the model you built for the Taxi Fare Prediction Problem using:
- Custom encoders for the distance and time features
- OneHot Encoder in order to encode the hour and day of week features
- SimpleImputer to fill missing values
- A simple linear regression
- A pipeline to put all together

Then: 
- train this pipeline
- apply the pipeline on test data
- generate predictions and submit these new predictions to Kaggle

## First pipeline

In [1]:
# import the train dataset ( 1000 first rows)
import pandas as pd
df = pd.read_csv('train.csv', nrows= 1000)
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
995,2011-07-30 07:33:13.0000001,16.9,2011-07-30 07:33:13 UTC,-73.884801,40.755707,-73.980472,40.765556,1
996,2011-09-24 23:20:00.000000112,13.7,2011-09-24 23:20:00 UTC,-73.953603,40.779203,-73.995763,40.726701,3
997,2011-12-22 11:07:00.00000037,29.3,2011-12-22 11:07:00 UTC,-73.942380,40.837712,-73.864372,40.769985,2
998,2014-11-03 12:40:00.00000014,6.5,2014-11-03 12:40:00 UTC,-73.961151,40.774578,-73.972251,40.785640,1


In [2]:
# prepare X and y
X= df.drop(["fare_amount"], axis=1)
y = df["fare_amount"]

In [3]:
# Hold out ( train and test dplit )
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.33,)

In [4]:
X_train.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
640,2013-02-15 17:36:00.00000075,2013-02-15 17:36:00 UTC,-74.002417,40.728325,-73.991863,40.744282,1
458,2010-01-09 21:28:55.0000003,2010-01-09 21:28:55 UTC,-73.952804,40.782491,-73.961038,40.774978,2
849,2011-08-08 09:59:00.00000093,2011-08-08 09:59:00 UTC,-73.962520,40.776573,-73.975502,40.757275,1
405,2014-02-06 11:17:00.00000089,2014-02-06 11:17:00 UTC,-73.954463,40.769887,-73.961627,40.779360,1
907,2014-06-20 01:02:07.0000004,2014-06-20 01:02:07 UTC,-73.995163,40.744714,-73.999739,40.728509,1


### Custom transformers

With the Taxi Fare Prediction Challenge data, using `BaseEstimator` and `TransformerMixin`, implement:

- a transformer that computes the haversine distance between the pickup and dropoff locations
- a custom encoder that extracts the time features from `pickup_datetime`

In [5]:
import numpy as np

def haversine_vectorized(df, 
                         start_lat="pickup_latitude",
                         start_lon="pickup_longitude",
                         end_lat="dropoff_latitude",
                         end_lon="dropoff_longitude"):
    """ 
        Calculates the great circle distance between two points 
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df.
        Computes the distance in kms.
    """

    lat_1_rad, lon_1_rad = np.radians(df[start_lat].astype(float)), np.radians(df[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(df[end_lat].astype(float)), np.radians(df[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [6]:
# create a DistanceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class DistanceTransformer(BaseEstimator, TransformerMixin):
    """
        Computes the haversine distance between two GPS points.
        Returns a copy of the DataFrame X with only one column: 'distance'.
    """

    def __init__(self,
                 start_lat="pickup_latitude",
                 start_lon="pickup_longitude",
                 end_lat="dropoff_latitude",
                 end_lon="dropoff_longitude"):
        # A COMPPLETER
        self.start_lat = start_lat
        self.start_lon = start_lon
        self.end_lat = end_lat
        self.end_lon = end_lon
    
    def fit(self, X, y=None):
        return self
        # A COMPLETER 

    def transform(self, X, y=None):
    
        return pd.DataFrame(haversine_vectorized(X)).rename(columns={0: "course distance [km]"}).copy()

In [7]:
# test the DistanceTransformer

dist_trans = DistanceTransformer()
distance = dist_trans.fit_transform(X_train, y_train)
distance.head()

,course distance [km]
640,1.984690
458,1.085627
849,2.408301
405,1.213861
907,1.842699


In [8]:
# create a TimeFeaturesEncoder
class TimeFeaturesEncoder(BaseEstimator, TransformerMixin):
    """
        Extracts the day of week (dow), the hour, the month and the year from a time column.
        Returns a copy of the DataFrame X with only four columns: 'dow', 'hour', 'month', 'year'.
    """

    def __init__(self, time_column, time_zone_name='America/New_York'):
        self.time_column = time_column
        self.time_zone_name = time_zone_name
       # A COMPLETER

    def extract_time_features(self, X):
        # A COMPLETER
        timezone_name = self.time_zone_name
        time_column = self.time_column
        df = X.copy()
        df.index = pd.to_datetime(df[time_column])
        df.index = df.index.tz_convert(timezone_name)
        df["dow"] = df.index.weekday
        df["hour"] = df.index.hour
        df["month"] = df.index.month
        df["year"] = df.index.year        
        return df
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return self.extract_time_features(X)[['dow', 'hour', 'month', 'year']].reset_index(drop=True)

In [9]:
# test the TimeFeaturesEncoder

time_enc = TimeFeaturesEncoder('pickup_datetime')
time_features = time_enc.fit_transform(X_train, y_train)
time_features.head()

,dow,hour,month,year
0,4,12,2,2013
1,5,16,1,2010
2,0,5,8,2011
3,3,6,2,2014
4,3,21,6,2014


###  Prepocessing pipeline

In [10]:
# visualizing pipelines in HTML
from sklearn import set_config; set_config(display='diagram')

#### Distance pipeline

Create a pipeline for distances:
- convert the pickup and dropoff coordinates into distances with the DistanceTransformer
- standardize these distances

In [11]:

# create distance pipeline dist_pipe
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipe_dist = Pipeline([
    ('distance', DistanceTransformer(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])),
    ('scaler',StandardScaler())
])
pipe_dist

# display distance pipeline


Pipeline(steps=[('distance',
                 DistanceTransformer(start_lat=['pickup_longitude',
                                                'pickup_latitude',
                                                'dropoff_longitude',
                                                'dropoff_latitude'])),
                ('scaler', StandardScaler())])

#### Time features pipeline

Create a pipeline for time features
- extract time features from pickup datetime with the TimeFeaturesEncoder
- encode these categorical time features with the OneHotEncoder

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
# create time pipeline time_pipe
pipe_time = Pipeline([
    ('time_features_create', TimeFeaturesEncoder('pickup_datetime')),
    ('time_features_ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])
pipe_time

# display time pipeline


Pipeline(steps=[('time_features_create',
                 TimeFeaturesEncoder(time_column='pickup_datetime')),
                ('time_features_ohe',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

#### Preprocessing pipeline

Wrap up the distance pipeline and the time pipeline into a preprocessing pipeline.

In [13]:
from sklearn.compose import ColumnTransformer

# create preprocessing pipeline preproc_pipe
preprocessor = ColumnTransformer([
    ('distance',pipe_dist, ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']),
    ('time', pipe_time,['pickup_datetime'])
])
preprocessor
# display preprocessing pipeline


ColumnTransformer(transformers=[('distance',
                                 Pipeline(steps=[('distance',
                                                  DistanceTransformer(start_lat=['pickup_longitude',
                                                                                 'pickup_latitude',
                                                                                 'dropoff_longitude',
                                                                                 'dropoff_latitude'])),
                                                 ('scaler', StandardScaler())]),
                                 ['pickup_longitude', 'pickup_latitude',
                                  'dropoff_longitude', 'dropoff_latitude']),
                                ('time',
                                 Pipeline(steps=[('time_features_create',
                                                  TimeFeaturesEncoder(time_column='pickup_datetime')),
                                                 ('time_features_ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['pickup_datetime'])])

### Model pipeline

Create a pipeline containing the preprocessing and the regression model of your choice.

In [14]:
from sklearn.linear_model import LinearRegression

# Add the model of your choice to the pipeline nammed pipe
final_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])
final_pipe
# display the pipeline with model


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('distance',
                                                  Pipeline(steps=[('distance',
                                                                   DistanceTransformer(start_lat=['pickup_longitude',
                                                                                                  'pickup_latitude',
                                                                                                  'dropoff_longitude',
                                                                                                  'dropoff_latitude'])),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['pickup_longitude',
                                                   'pickup_latitude',
                                                   'dropoff_longitude',
                                                   'dropoff_latitude']),
                                                 ('time',
                                                  Pipeline(steps=[('time_features_create',
                                                                   TimeFeaturesEncoder(time_column='pickup_datetime')),
                                                                  ('time_features_ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['pickup_datetime'])])),
                ('model', LinearRegression())])

### Training and performance

Train the pipelined model and compute the prediction on the test set:

In [15]:
# train the pipelined model
pipeline = final_pipe

pipeline.fit(X_train, y_train)

# compute y_pred on the test set
y_pred = pipeline.predict(X_test)

C:\Users\utilisateur\anaconda3\envs\nainporte\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Use the RMSE to evaluate the performance of the model:

In [16]:
def compute_rmse(y_pred, y_true):
    
    return np.sqrt(np.mean(y_true - y_pred)**2)

In [17]:
# call compute_rmse
compute_rmse(y_pred, y_test)

0.7396458333333334

## Complete the workflow with a pipeline

Here we will implement the whole workflow for our Taxifare kaggle challenge.

For that we will refactor the code in functions for more clarity.

Implement the following functions:
- `get_data()` to fetch the data 
- `clean_data()` to clean the data
- `get_pipeline()` to get the pipeline defined earlier
- `train()` to train our model
- `evaluate()` to evaluate our model on test data

In [18]:
# implement get_data() function
def get_data(nrows=10000):
    '''returns a DataFrame with nrows from s3 bucket'''
    # Load the data into a pandas DataFrame
    df = pd.read_csv('train.csv', nrows=nrows)
    return df

get_data().head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [19]:
# implement clean_data() function
def clean_data(df, test=False):
    '''returns a DataFrame without outliers and missing values'''
    # A COMPLETER
    df = df.drop(["key"], axis=1)
    df = df.dropna(how='any', axis='rows')
    df = df[(df.dropoff_latitude != 0) | (df.dropoff_longitude != 0)]
    df = df[(df.pickup_latitude != 0) | (df.pickup_longitude != 0)]
    if "fare_amount" in list(df):
        df = df[df.fare_amount.between(0, 4000)]
        df = df[df.passenger_count < 8]
        df = df[df.passenger_count >= 0]
        df = df[df["pickup_latitude"].between(left=40, right=42)]
        df = df[df["pickup_longitude"].between(left=-74.3, right=-72.9)]
        df = df[df["dropoff_latitude"].between(left=40, right=42)]
        df = df[df["dropoff_longitude"].between(left=-74, right=-72.9)]
    return df
data_train = clean_data(df)
data_train.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [20]:
# implement set_pipeline() function
def set_pipeline():
    '''returns a pipelined model'''
     # A  COMPLETER
    pipe_dist = Pipeline([('distance', DistanceTransformer()), 
                          ('scaler', StandardScaler())])
    pipe_time = Pipeline([('time_features_create', TimeFeaturesEncoder('pickup_datetime')), 
                          ('time_features_ohe', OneHotEncoder(handle_unknown='ignore'))])
    preprocessor = ColumnTransformer([('distance', 
                                       pipe_dist, 
                                       ["pickup_latitude", 
                                        "pickup_longitude", 
                                        'dropoff_latitude', 
                                        'dropoff_longitude']), 
                                      ('time', 
                                       pipe_time, 
                                       ['pickup_datetime'])], 
                                     remainder="drop")
    final_pipe = Pipeline([('preprocessor', preprocessor),
                    ('model', LinearRegression())])
    return final_pipe

In [21]:
# implement train() function
def train(X_train, y_train, pipeline):
    '''returns a trained pipelined model'''
   # A COMPLETER
    pipeline.fit(X_train, y_train)
    return pipeline

In [22]:
# implement evaluate() function
def evaluate(X_test, y_test, pipeline):
    '''returns the value of the RMSE'''
    # A COMPLETER
    y_pred = pipeline.predict(X_test)
    rmse = compute_rmse(y_pred, y_test)

    print(rmse)
    return rmse

### Test the complete worflow

Use the above functions to test the complete workflow.

In [23]:
# store the data in a DataFrame
df = get_data()

# set X and y
y = df["fare_amount"]
X = df.drop("fare_amount", axis=1)

# hold out
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

# build pipeline
pipeline = set_pipeline()

# train the pipeline
train(X_train, y_train, pipeline)

# evaluate the pipeline
rmse = evaluate(X_val, y_val, pipeline)

0.2089923342746882


### Congrats!

Now we are ready to convert this complete workflow into a packaged code 🚀